**Giang Pham**

Spring 2023

CS 251/2: Data Analysis and Visualization

Project 6: Supervised learning

In [31]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=5)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Task 3: Preprocess full spam email dataset 

Before you build a Naive Bayes spam email classifier, run the full spam email dataset through your preprocessing code.

Download and extract the full **Enron** emails (*zip file should be ~29MB large*). You should see a base `enron` folder, with `spam` and `ham` subfolders when you extract the zip file (these are the 2 classes).

Run the test code below to check everything over.

### 3a) Preprocess dataset

In [32]:
import email_preprocessor as epp

#### Test `count_words` and `find_top_words`

In [33]:
word_freq, num_emails = epp.count_words("data/enron/")

In [34]:
print(f'You found {num_emails} emails in the datset. You should have found 32625.')

You found 32625 emails in the datset. You should have found 32625.


In [35]:
top_words, top_counts = epp.find_top_words(word_freq)
print(f"Your top 5 words are\n{top_words[:5]}\nand they should be\n['the', 'to', 'and', 'of', 'a']")
print(f"The associated counts are\n{top_counts[:5]}\nand they should be\n[277459, 203659, 148873, 139578, 111796]")

Your top 5 words are
['the', 'to', 'and', 'of', 'a']
and they should be
['the', 'to', 'and', 'of', 'a']
The associated counts are
[277459, 203659, 148873, 139578, 111796]
and they should be
[277459, 203659, 148873, 139578, 111796]


### 3b) Make train and test splits of the dataset

Here we divide the email features into a 80/20 train/test split (80% of data used to train the supervised learning model, 20% we withhold and use for testing / prediction).

In [36]:
features, y = epp.make_feature_vectors(top_words, num_emails)

In [37]:
np.random.seed(0)
x_train, y_train, inds_train, x_test, y_test, inds_test = epp.make_train_test_sets(features, y)

In [38]:
print('Shapes for train/test splits:')
print(f'Train {x_train.shape}, classes {y_train.shape}')
print(f'Test {x_test.shape}, classes {y_test.shape}')
print('\nThey should be:\nTrain (26100, 200), classes (26100,)\nTest (6525, 200), classes (6525,)')

Shapes for train/test splits:
Train (26100, 200), classes (26100,)
Test (6525, 200), classes (6525,)

They should be:
Train (26100, 200), classes (26100,)
Test (6525, 200), classes (6525,)


### 3c) Save data in binary format

It adds a lot of overhead to have to run through your raw email -> train/test feature split every time you wanted to work on your project! In this step, you will export the data in memory to disk in a binary format. That way, you can quickly load all the data back into memory (directly in ndarray format) whenever you want to work with it again. No need to parse from text files!

Running the following cell uses numpy's `save` function to make six files in `.npy` format (e.g. `email_train_x.npy`, `email_train_y.npy`, `email_train_inds.npy`, `email_test_x.npy`, `email_test_y.npy`, `email_test_inds.npy`).

In [39]:
np.save('data/email_train_x.npy', x_train)
np.save('data/email_train_y.npy', y_train)
np.save('data/email_train_inds.npy', inds_train)
np.save('data/email_test_x.npy', x_test)
np.save('data/email_test_y.npy', y_test)
np.save('data/email_test_inds.npy', inds_test)

## Task 4: Naive Bayes Classifier

After finishing your email preprocessing pipeline, implement the one other supervised learning algorithm we we will use to classify email, **Naive Bayes**.

### 4a) Implement Naive Bayes

In `naive_bayes.py`, implement the following methods:
- Constructor
- get methods
- `train(data, y)`: Train the Naive Bayes classifier so that it records the "statistics" of the training set: class priors (i.e. how likely an email is in the training set to be spam or ham?) and the class likelihoods (the probability of a word appearing in each class — spam or ham).
- `predict(data)`: Combine the class likelihoods and priors to compute the posterior distribution. The predicted class for a test sample is the class that yields the highest posterior probability.
- `accuracy(y, y_pred)`: The usual definition :)


#### Bayes rule ingredients: Priors and likelihood (`train`)

To compute class predictions (probability that a test example belong to either spam or ham classes), we need to evaluate **Bayes Rule**. This means computing the priors and likelihoods based on the training data.

**Prior:** $$P_c = \frac{N_c}{N}$$ where $P_c$ is the prior for class $c$ (spam or ham), $N_c$ is the number of training samples that belong to class $c$ and $N$ is the total number of training samples.

**Likelihood:** $$L_{c,w} = \frac{N_{c,w} + 1}{N_{c} + M}$$ where
- $L_{c,w}$ is the likelihood that word $w$ belongs to class $c$ (*i.e. what we are solving for*)
- $N_{c,w}$ is the total count of **word $w$** in emails that are only in class $c$ (*either spam or ham*)
- $N_{c}$ is the total number of **all words** that appear in emails of the class $c$ (*total number of words in all spam emails or total number of words in all ham emails*)
- $M$ is the number of features (*number of top words*).

#### Bayes rule ingredients: Posterior (`predict`)

To make predictions, we now combine the prior and likelihood to get the posterior:

**Log Posterior:** $$Log(\text{Post}_{i, c}) = Log(P_c) + \sum_{j \in J_i}x_{i,j}Log(L_{c,j})$$

 where
- $\text{Post}_{i,c}$ is the posterior for class $c$ for test sample $i$(*i.e. evidence that email $i$ is spam or ham*). We solve for its logarithm.
- $Log(P_c)$ is the logarithm of the prior for class $c$.
- $x_{i,j}$ is the number of times the jth word appears in the ith email.
- $Log(L_{c,j})$: is the log-likelihood of the jth word in class $c$.

In [40]:
from naive_bayes import NaiveBayes

#### Test `train`

###### Class priors and likelihoods

The following test should be used only if storing the class priors and likelihoods directly.

In [41]:
num_test_classes = 4
np.random.seed(0)
data_test = np.random.randint(low=0, high=20, size=(100, 6))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_test, y_test)

print(f'Your class priors are: {nbc.get_priors()}\nand should be          [0.28 0.22 0.32 0.18].')
print(f'Your class likelihoods shape is {nbc.get_likelihoods().shape} and should be (4, 6).')
print(f'Your likelihoods are:\n{nbc.get_likelihoods()}')

print(f'and should be')
print('''[[0.15997 0.15091 0.2079  0.19106 0.14184 0.14832]
 [0.11859 0.16821 0.17914 0.16905 0.18082 0.18419]
 [0.16884 0.17318 0.14495 0.14332 0.18784 0.18187]
 [0.16126 0.17011 0.15831 0.13963 0.18977 0.18092]]''')

Your class priors are: [0.28 0.22 0.32 0.18]
and should be          [0.28 0.22 0.32 0.18].
Your class likelihoods shape is (4, 6) and should be (4, 6).
Your likelihoods are:
[[0.15997 0.15091 0.2079  0.19106 0.14184 0.14832]
 [0.11859 0.16821 0.17914 0.16905 0.18082 0.18419]
 [0.16884 0.17318 0.14495 0.14332 0.18784 0.18187]
 [0.16126 0.17011 0.15831 0.13963 0.18977 0.18092]]
and should be
[[0.15997 0.15091 0.2079  0.19106 0.14184 0.14832]
 [0.11859 0.16821 0.17914 0.16905 0.18082 0.18419]
 [0.16884 0.17318 0.14495 0.14332 0.18784 0.18187]
 [0.16126 0.17011 0.15831 0.13963 0.18977 0.18092]]


###### Log of class priors and likelihoods

This test should be used only if storing the log of the class priors and likelihoods.

In [42]:
num_test_classes = 4
np.random.seed(0)
data_test = np.random.randint(low=0, high=20, size=(100, 6))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_test, y_test)

print(f'Your log class priors are: {nbc.get_priors()}\nand should be              [-1.27297 -1.51413 -1.13943 -1.7148 ].')
print(f'Your log class likelihoods shape is {nbc.get_likelihoods().shape} and should be (4, 6).')
print(f'Your log likelihoods are:\n{nbc.get_likelihoods()}')


print(f'and should be')
print('''[[-1.83274 -1.89109 -1.57069 -1.65516 -1.95306 -1.90841]
 [-2.13211 -1.78255 -1.71958 -1.77756 -1.71023 -1.6918 ]
 [-1.77881 -1.75342 -1.93136 -1.94266 -1.67217 -1.70448]
 [-1.82475 -1.77132 -1.84321 -1.96879 -1.66192 -1.70968]]''')

Your log class priors are: [0.28 0.22 0.32 0.18]
and should be              [-1.27297 -1.51413 -1.13943 -1.7148 ].
Your log class likelihoods shape is (4, 6) and should be (4, 6).
Your log likelihoods are:
[[0.15997 0.15091 0.2079  0.19106 0.14184 0.14832]
 [0.11859 0.16821 0.17914 0.16905 0.18082 0.18419]
 [0.16884 0.17318 0.14495 0.14332 0.18784 0.18187]
 [0.16126 0.17011 0.15831 0.13963 0.18977 0.18092]]
and should be
[[-1.83274 -1.89109 -1.57069 -1.65516 -1.95306 -1.90841]
 [-2.13211 -1.78255 -1.71958 -1.77756 -1.71023 -1.6918 ]
 [-1.77881 -1.75342 -1.93136 -1.94266 -1.67217 -1.70448]
 [-1.82475 -1.77132 -1.84321 -1.96879 -1.66192 -1.70968]]


#### Test `predict`

In [43]:
num_test_classes = 4
np.random.seed(0)
data_train = np.random.randint(low=0, high=num_test_classes, size=(100, 10))
data_test = np.random.randint(low=0, high=num_test_classes, size=(15, 10))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_train, y_test)
test_y_pred = nbc.predict(data_test)

print(f'Your predicted classes are\n{test_y_pred}\nand should be\n[3 0 3 1 0 1 1 3 0 3 0 2 0 2 1]')

Your predicted classes are
[3 0 3 1 0 1 1 3 0 3 0 2 0 2 1]
and should be
[3 0 3 1 0 1 1 3 0 3 0 2 0 2 1]


### 4b) Spam filtering

Let's start classifying spam email using the Naive Bayes classifier. The following code uses `np.load` to load in the train/test split that you created last week.
- Use your Naive Bayes classifier on the Enron email dataset!

**Question 7:** Print out the accuracy that you get on the test set with Naive Bayes. It should be roughly 89%.

In [44]:
import email_preprocessor as ep

In [45]:
x_train = np.load('data/email_train_x.npy')
y_train = np.load('data/email_train_y.npy')
inds_train = np.load('data/email_train_inds.npy')
x_test = np.load('data/email_test_x.npy')
y_test = np.load('data/email_test_y.npy')
inds_test = np.load('data/email_test_inds.npy')

In [46]:
from naive_bayes import NaiveBayes

nbenron = NaiveBayes(num_classes=2)
nbenron.train(x_train, y_train)
y_pred = nbenron.predict(x_test)
acc_enron = nbenron.accuracy(y_test, y_pred=y_pred)
print(acc_enron)
print(y_pred.shape)

0.8924137931034483
(6525,)


### 4c) Confusion matrix

To get a better sense of the errors that the Naive Bayes classifer makes, you will create a confusion matrix. 

- Implement `confusion_matrix` in `naive_bayes.py`.
- Print out a confusion matrix of the spam classification results.

**Debugging guidelines**:
1. The sum of all numbers in your 2x2 confusion matrix should equal the number of test samples (6525).
2. The sum of your spam row should equal the number of spam samples in the test set (3193)
3. The sum of your ham row should equal the number of spam samples in the test set (3332)

In [47]:
confusion = nbenron.confusion_matrix(y_test, y_pred)
print("Confusion matrix, row 0 is ham, row 1 is spam")
print(confusion)
#for debugging
print("total samples: ", np.sum(confusion))
print("total ham:", np.sum(confusion[0]))
print("total spam:", np.sum(confusion[1]))

Confusion matrix, row 0 is ham, row 1 is spam
[[2780.  552.]
 [ 150. 3043.]]
total samples:  6525.0
total ham: 3332.0
total spam: 3193.0


**Question 8:** Interpret the confusion matrix, using the convention that positive detection means spam (*e.g. a false positive means classifying a ham email as spam*). What types of errors are made more frequently by the classifier? What does this mean (*i.e. X (spam/ham) is more likely to be classified than Y (spam/ham) than the other way around*)?

**Reminder:** Look back and make sure you are clear on which class indices correspond to spam/ham.

**Answer 8:**
We can see that there are more false positives then false negatives (552 versus 150). This mean that more ham emails are classfied as spams

## Task 5: Comparison with KNN


- Run a similar analysis to what you did with Naive Bayes above. When computing accuracy on the test set, you may want to reduce the size of the test set (e.g. to the first 500 emails in the test set).
- Copy-paste your `confusion_matrix` method into `knn.py` so that you can run the same analysis on a KNN classifier.

In [48]:
from knn import KNN

In [49]:
#reduce the size of knn test set
x_test_knn = np.random.permutation(x_test[:500,:])
y_test_knn = np.random.permutation(y_test[:500])
x_train_knn  = np.random.permutation(x_train)
y_train_knn = np.random.permutation(y_train)
print(x_test_knn.shape)
print(y_test_knn.shape)

#run analysis on KNN 
knnenron = KNN(num_classes=2)
knnenron.train(x_train_knn, y_train_knn)
y_pred_knn = knnenron.predict(x_test_knn, 10)
acc_enron_knn = knnenron.accuracy(y_test_knn, y_pred_knn)
print(acc_enron_knn)

print("KNN Confusion matrix, row 0 is ham, row 1 is spam")
print(knnenron.confusion_matrix(y_pred_knn,y_test_knn))

(500, 200)
(500,)
0.498
KNN Confusion matrix, row 0 is ham, row 1 is spam
[[148. 166.]
 [ 85. 101.]]


**Question 9:** What accuracy did you get on the test set (potentially reduced in size)?

**Question 10:** How does the confusion matrix compare to that obtained by Naive Bayes (*If you reduced the test set size, keep that in mind*)?

**Question 11:** Briefly describe at least one pro/con of KNN compared to Naive Bayes on this dataset.

**Question 12:** When potentially reducing the size of the test set here, why is it important that we shuffled our train and test set?

**Answer 9:** 
I got a 90.4 percent accuracy, as shown in the code above. 

**Answer 10:** 
KNN: 
False positive portion: 178/500 = 0.356
False negative portion: 85/500 = 0.17
-> Total false portion: 0.526

Naive Bayes: 
False positive portion: 552/6525 = 0.084
False negative portion: 150/6525 = 0.023
-> Total false portion: 0.107

The first thing we can see from the confusion matrices of 2 algorithms is that KNN has more false positive than false negatives, which is the same as Naive Bayes. From the calculations above, we also see that both the false positive and false negative portion of Naive Bayes is less than that of KNN, so NB performs a better job.

**Answer 11:** 
Pro of KNN: KNN remembers all the training data and so it doesn't make any assumptions of the dataset, while the NB assumes that certain words are independent of each other, which may potentially lead to wrong answer. 

A con of KNN is that as seen from the calculations, remembering literal exemplars might not be the best way to classify emails, leading to very low accuaracy. KNN might work well for numeric data, but in this case, KNN might not be the best fit for word data like email classification. Moreover, KNN is also a lot slower than NB. 

**Answer 12:** 
We need to shuffle the train and test set so that our dataset are not biased if the data are sorted in someway that have all the beginning samples belong to the same class or have some same characteristic. It makes the data more general.

## Extensions

### 6. Investigate the misclassification errors

Numbers are nice, but they may not the best for developing your intuition. Sometimes, you want to see what an misclassification *actually looks like* to help you improve your algorithm. Retrieve the actual text of some example emails of false positive and false negative misclassifications to see if helps you understand why the misclassification occurred. Here is an example workflow:

- Decide on how many FP and FN emails you would like to retrieve. Find the indices of this many false positive and false negative misclassification. Remember to use your `test_inds` array to look up the index of the emails BEFORE shuffling happened.
- Implement the function `retrieve_emails` in `email_preprocessor.py` to return the string of the raw email at the error indices.
- Call your function to print out the emails that produced misclassifications.

Do the FP and FN emails make sense? Why? Do the emails have properties in common? Can you quantify and interpret them?

For this extension, I implemented the retrieve_emails method in email_preprocessor.py. I also implemented a helper function get_false_indices in naive_bayes.py that takes in y and y_pred and return a list of indices of emails that are falsely classified (either false positive or false negative). These indices are then compared with inds_test to return the real indices of emails before shuffling. These final indices are passed into retrieve_emails to then return the full string of the falsely classified emails.

In [50]:
# test retrieve emails
print(epp.retrieve_emails(inds=[0,1,4], email_path="data/enron_dev"))
print(epp.retrieve_emails(inds=[0,1,20], email_path="data/enron_dev"))
print(epp.retrieve_emails(inds=[0,2,2], email_path="data/enron_dev"))

['Subject: get that new car now\n', 'Subject: you can be smart !\n', 'Subject: re : rankings\nthank you .']
['Subject: get that new car now\n', 'Subject: you can be smart !\n']
['Subject: get that new car now\n', 'Subject: love ecards\n']


In [51]:
# number of false positve and false negative emails to retrieve
n_fp = 3
n_fn = 3

#compute the indices of fp and fn email after shuffling
fp_shuffled, fn_shuffled = nbenron.get_false_indices(y_test, y_pred)

#compute the indices of fp and fn email before shuffling
fp_true_ind = np.array([inds_test[i] for i in fp_shuffled])
fn_true_ind = np.array([inds_test[i] for i in fn_shuffled])

#testing the shape of fp and fn
print(fp_true_ind.shape)
print(fn_true_ind.shape)

6525
(552,)
(150,)


In [57]:
#choosing random indices from the index list
random_fp_indices = np.random.choice(len(fp_true_ind), size = n_fp, replace=False)

print("False positive emails")
#get the list of randomly chosen fp emails 
fps = epp.retrieve_emails(fp_true_ind[random_fp_indices])
for fp in fps: 
    print(fp, "\n")

False positive emails
Subject: tufco
my est . for tufco is the same as last may ' s ( 1999 ) nom .
iferc 40000 / d total of 1 , 240 , 000
wb 30000 / d total of 930 , 000
ami 

Subject: dp question
jeff :
you asked how your gross margin % could be higher in 2000 but your overall
gross margin be lower .
you appear to have lower invoiced volumes in jan - april 2000 .
january and feb . - for example :
great question !
dave 

Subject: start date : 1 / 27 / 02 ; hourahead hour : 21 ;
start date : 1 / 27 / 02 ; hourahead hour : 21 ; no ancillary schedules awarded . no variances detected .
log messages :
parsing file - - > > o : \ portland \ westdesk \ california scheduling \ iso final schedules \ 2002012721 . txt 



In [58]:
random_fn_indices = np.random.choice(len(fn_true_ind), size = n_fn, replace=False)

print("False negative emails")
#get the list of randomly chosen fp emails 
fns = epp.retrieve_emails(fn_true_ind[random_fn_indices])
for fn in fns: 
    print(fn, "\n")

False negative emails
Subject: crunch force leaden d ' s bovine diatomaceous pegasus fortune billionth infinity beryl food el cheesecloth benz coralline lyricism brim detractor duff antoine gantlet
female dosage goldberg bayonne emphysema erickson hydrogenate grandstand crackpot game polytypy octahedral celtic dibble crisis azerbaijan gigavolt fermat cavilling invariant examine hom gould charybdis database beggary heal heroic backstop catherine inapplicable rainy extol fulton brontosaurus contextual
vibgra for woemen
schenectady mutineer poseidon excretion greyhound gar albany inexpedient kalmia attest buzz euthanasia gripe cottonmouth borosilicate ground dapple bitch pacifism ductile scourge finicky freehand
 

Subject: = ? utf - 8 ? q ? xmas wishlist - role ? =
= ? utf - 8 ? q ? x replica ? =
genuine replications watches
we offer the following trade names available as well :
harry winston
breguet
glashute original
officine panerai
tag heuer
 

Subject: returned mail : see transcript 

Analyze: Readind the emails that are retrieved, I can see some features that might make them be potentially be misclassified. 

In the list of false positve emails (ham classified as spam), I can notice some keywords that can make the program think that these are spam. For examples, "payment", "visit our websites", commercial-like information such as "what is livegraphic 3d" when it is actually just inteded for context of the main email, or "options pricing", which sounds like a ad to me. 

False negative emails are harder to pinpoint why they are classified as ham. I can see that a lot of them has casual conversational language instead of using advertisement-like terms, which might explain why they are classified as ham.